<a href="https://colab.research.google.com/github/nug1209/PwC_Switzerland_Digital_Intelligence_Virtual_Case_Experience/blob/main/PwC_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this task we deal with merchant loan data. We will create a forecast about how much will the total repayment will be after these loans are finished (after 30 months since the loans started). The formula for counting the repayment is provided. We will also create percentage values for each repayment in comparison with the origination amount.

Import libraries.

In [247]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from datetime import datetime

Mount Google Drive to get the data.

In [248]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Virtual Experience/PwC_Task_2_Data.csv'

df = pd.read_csv(path, delimiter=';')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Rename columns.

In [249]:
df = df.rename(columns={'Unnamed: 0':'origination_date', 'Origination Amount':'origination_amount'})
df.head(1)

,origination_date,origination_amount,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,...,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
0,31.05.2019,10018746.17,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,...,116684.68,92699.67,63399.66,53265.12,37121.13,29787.1,24524.9,18085.94,16581.01,11442.97


In [250]:
df.shape

(20, 22)

Preparation for melting, i.e. transform wide format table to long format.

In [251]:
to_remove = ['origination_date', 'origination_amount']
value_columns = [i for i in list(df.columns) if i not in to_remove]

This was to calculate repayment percentage early in the processing, but may be it is better done later.

In [252]:
# for i in value_columns:
#   df[i + '_actual_repayment_percent'] = 100 * df[i] / df['origination_amount']

# df.columns

Convert to long format.

In [253]:
df = pd.melt(df, id_vars=['origination_date', 'origination_amount'], value_vars=value_columns)
df = df.rename(columns={'variable':'repayment_date', 'value':'repayment_amount'})

Remove dates with no repayment.

In [254]:
df = df[df['repayment_amount'] != 0.00]
# df

In [255]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 0 to 399
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origination_date    210 non-null    object 
 1   origination_amount  210 non-null    float64
 2   repayment_date      210 non-null    object 
 3   repayment_amount    210 non-null    float64
dtypes: float64(2), object(2)
memory usage: 8.2+ KB


Add one row for the last entry as instructed in the task.

In [256]:
add_row = {'origination_date':['31.12.2020'], 'origination_amount':[30482978.52], 'repayment_date':['31.01.2021'], 'repayment_amount':[8747661.94]}

df = pd.concat([df, pd.DataFrame(add_row)], ignore_index=True)

In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origination_date    211 non-null    object 
 1   origination_amount  211 non-null    float64
 2   repayment_date      211 non-null    object 
 3   repayment_amount    211 non-null    float64
dtypes: float64(2), object(2)
memory usage: 6.7+ KB


Convert the date columns to date type.

In [258]:
df['origination_date'] = pd.to_datetime(df['origination_date'], format='%d.%m.%Y')
df['repayment_date'] = pd.to_datetime(df['repayment_date'], format='%d.%m.%Y')
# df.info()

Sort values.

In [259]:
df = df.sort_values(by=['origination_date', 'repayment_date'])
df

,origination_date,origination_amount,repayment_date,repayment_amount
0,2019-05-31,10018746.17,2019-05-31,1443069.08
1,2019-05-31,10018746.17,2019-06-30,3332200.33
3,2019-05-31,10018746.17,2019-07-31,1328138.75
6,2019-05-31,10018746.17,2019-08-31,928085.74
10,2019-05-31,10018746.17,2019-09-30,736418.27
...,...,...,...,...
207,2020-10-31,27699586.46,2020-12-31,1503544.68
189,2020-11-30,29872889.68,2020-11-30,4383982.78
208,2020-11-30,29872889.68,2020-12-31,8383025.07
209,2020-12-31,30482978.52,2020-12-31,4373830.97


In [260]:
df_copy = df.copy()

Create data frame from unique values of origination date and origination amount so that it can be merged later with the repayment values.

In [261]:
df1 = df[['origination_date', 'origination_amount']].drop_duplicates().reset_index(drop=True)
# df1

Get the first two repayments.

In [262]:
two_repayments = df.groupby('origination_date')['repayment_amount'].apply(list).reset_index(name='repayment')['repayment'].apply(lambda x: x[0: 2])
df2 = pd.DataFrame(two_repayments, columns=['repayment'])
# df2

In [263]:
df_agg = pd.concat([df1, df2], axis=1)
# df_agg

In [264]:
# def repayment_percent(a, b):
#   return [100* i / b for i in a]

def repayment_part(a, b):
  return [i / b for i in a]

In [265]:
df_agg['repayment_part'] = df_agg.apply(lambda row: repayment_part(row['repayment'], row['origination_amount']), axis=1)
df_agg

,origination_date,origination_amount,repayment,repayment_part
0,2019-05-31,10018746.17,"[1443069.08, 3332200.33]","[0.14403689398989936, 0.33259654186847215]"
1,2019-06-30,10868379.04,"[1392751.6, 3011884.91]","[0.1281471316811932, 0.277123653758767]"
2,2019-07-31,10733932.61,"[1537650.24, 2953335.55]","[0.14325134094539466, 0.2751401240630669]"
3,2019-08-31,12558727.02,"[1617681.94, 4082016.0]","[0.128809387880142, 0.32503421672429983]"
4,2019-09-30,14505071.44,"[1992242.84, 3930445.6]","[0.13734801984539555, 0.27097044066678516]"
5,2019-10-31,15652952.20,"[2289453.76, 4682354.31]","[0.14626338410462916, 0.2991355400676429]"
6,2019-11-30,15107713.30,"[2162283.09, 4637701.69]","[0.14312444557708145, 0.30697575456373005]"
7,2019-12-31,17004745.04,"[2402403.37, 4947764.21]","[0.14127841166385405, 0.29096373973037826]"
8,2020-01-31,16794379.95,"[2502066.86, 4696910.48]","[0.14898238979046083, 0.27967156239072705]"
9,2020-02-29,19217205.82,"[2833811.35, 6142911.08]","[0.1474621948967605, 0.3196568292777956]"


In [266]:
def expected_repayment_part(p):
  
  for i in np.arange(3, 31):
        
    calc1 = sum(p[:-1])
    
    calc2 = 1 - calc1
    
    calc3 = (i - 1) / 30
    
    calc4 = 1 - calc2
    
    calc5 = 1 + (calc4 * calc2)
    
    calc6 = p[1] * np.log(calc5)
    
    pi = max(calc6, 0)
    
    # pi = 99.79 * pi

    p.append(pi)

    # i += 1
  
  return p
	

In [267]:
## This is a version of the repayment formula that is equipped with print functions for debugging purposes. I leave it here for now.

# def expected_repayment_part(p):
  
#   for i in np.arange(3, 30):
    
#     print('i', i)
    
#     calc1 = sum(p[:-1])
#     print('calc1', calc1)

#     calc2 = 1 - calc1
#     print('calc2', calc2)

#     calc3 = (i - 1) / 30
#     print('calc3', calc3)

#     calc4 = 1 - calc2
#     print('calc4', calc4)

#     calc5 = 1 + (calc4 * calc2)
#     print('calc5', calc5)

#     calc6 = p[1] * np.log(calc5)
#     print('calc6', calc6)

#     pi = max(calc6, 0)
#     print('pi', pi)

#     # pi = 99.79 * pi

#     p.append(pi)

#     i += 1
  
#   return p
	

In [268]:
# p = [0.14403689398989936, 0.33259654186847215]
# expected_repayment_part(p)

In [269]:
df_agg['expected_repayment_part'] = df_agg['repayment_part'].apply(expected_repayment_part)

In [279]:
df_agg['total_expected_repayment_part'] = df_agg['expected_repayment_part'].apply(sum)

In [281]:
df_agg['total_expected_repayment'] = df_agg['total_expected_repayment_part'] * df_agg['origination_amount']

In [283]:
df_agg['total_expected_repayment_after_discount'] = df_agg['total_expected_repayment'] * 97.5 * 30 / 12

In [285]:
df_agg['margin'] = df_agg['total_expected_repayment_after_discount'] - df_agg['origination_amount']

In [299]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
df_result = df_agg[['origination_date', 'origination_amount', 'total_expected_repayment_part', 'total_expected_repayment_after_discount', 'margin']].round(2)
# df_result.apply(lambda x: '%.5f % x, axis=1')
df_result.head(3)

,origination_date,origination_amount,total_expected_repayment_part,total_expected_repayment_after_discount,margin
0,2019-05-31,10018746.17000,1.00000,2447344436.65000,2437325690.48000
1,2019-06-30,10868379.04000,1.00000,2649312267.35000,2638443888.31000
2,2019-07-31,10733932.61000,1.00000,2616498791.71000,2605764859.10000


Sanity check.

In [297]:
2447344436.65 - 10018746.17	

2437325690.48